<a href="https://colab.research.google.com/github/MagdaPla/GEE_RemoteSensing/blob/main/GEE_Landsat_Estepiques_tot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# muntar el directori de drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Run the following cell to import the API into your session.
import ee
# import other packs
import os 
import datetime 
from datetime import datetime as dt
import sys
from collections import namedtuple

In [ ]:
#Create structure to hold setting parameters
set_par_est = namedtuple("set_par_est", "sel_col bands p_out offset QA QA_bands Indices scale mult")

Ara introduïm algunes funcions per a enmascarar núvols de Landsat, per a tractar les dates, etc.

In [ ]:
#https://gis.stackexchange.com/questions/274048/apply-cloud-mask-to-landsat-imagery-in-google-earth-engine-python-api
#A function to mask out cloudy pixels.
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end-1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)


#https://gis.stackexchange.com/questions/274048/apply-cloud-mask-to-landsat-imagery-in-google-earth-engine-python-api
def maskQuality(image):
    # Select the QA band.
    QA = image.select('pixel_qa')
    # Get the internal_cloud_algorithm_flag bit.
    sombra = getQABits(QA,3,3,'cloud_shadow')
    nubes = getQABits(QA,5,5,'cloud')
    #  var cloud_confidence = getQABits(QA,6,7,  'cloud_confidence')
    cirrus_detected = getQABits(QA,9,9,'cirrus_detected')
    snow_detected = getQABits(QA,4,4,'snow_detected')
    #var cirrus_detected2 = getQABits(QA,8,8,  'cirrus_detected2')
    #Return an image masking out cloudy areas.
    return image.updateMask(sombra.eq(0)).updateMask(nubes.eq(0).updateMask(cirrus_detected.eq(0))).updateMask(snow_detected.eq(0))


In [ ]:
#This procedure computes the number of processing periods depending on a temporal extent and a time interval given
#If the processing periods are not an interger number, meaning an inrregural time series, then the last processing period 
#will become longer including more dates, thus turning the time series into a regular time series
#It returns a string array with all the processing periods

def ComputeProcessingPeriods(time, time_int):

    #date conversion
    d_start=datetime.datetime.strptime(str(time[0]), '%Y-%m-%d')
    d_end=datetime.datetime.strptime(str(time[1]), '%Y-%m-%d')
    
    #total number of days to be processed
    n_days=(d_end-d_start).days+1
    print('Number of days to be processed: '+str(n_days))
    
    #number of processing periods
    print('Number of processing periods: '+str(n_days/time_int))

    #array storing data    
    proc_per=[]

    #Make sure we have regular processing periods.
    #If not assign the remaining days to the last period
    if (n_days%time_int == 0): #In this case the time series is regular
            
        print('Time series is regular: total number of processing periods= '+str(n_days/time_int))
        
        n_pp=int(n_days/time_int)
        
        for i in range(n_pp):

            #set timedelta
            td=time_int*(i+1)-1
    
            date_s=str(d_start+datetime.timedelta(days=time_int*(i))).split(' ')
            date_e=str(d_start+datetime.timedelta(days=time_int*(i+1))).split(' ')
            
            proc_per.append([date_s[0],date_e[0]])
        
    else: #Time series not regular
            
        reg=0   

  
    return proc_per

A continuació definim la col·lecció de dades amb les que volem treballar, aquest codi està preparat per a diferents productes de Modis i Landsat

In [ ]:
def CollectionSelectionPar(option):
    
    if option ==1: #Collection NDVI 500 m
        
        #product selection
        sel_col='MODIS/MOD09GA_006_NDVI'
        #bands to be processed
        bands = ['NDVI']
        #Google drve output folder
        p_out='UIB-Imatges-MOD09GA-006-NDVI'
        #Image offset
        offset=''


    elif option ==2: #Collection EVI 500 m

        sel_col='UIB-Imatges-MOD09GA-006-EVI'
        bands = ['EVI']
        p_out='UIB-Imatges-MOD09GA_006_EVI'
        #Image offset
        offset=''

    elif option ==3: #Collection surface reflectance 250 m

        sel_col='MODIS/006/MOD09GQ'
        bands = ['sur_refl_b01', 'sur_refl_b02']
        #Â¡p_out='UIB-Imatges-MOD09GQ'
        p_out='UIB-Imatges-MOD09GQ'
        #Image offset
        offset=10000
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA='MODIS/006/MOD09GA'
        QA_bands = ['state_1km', 'QC_500m']
        Indices=['NDVI','EVI2']
        scale=250
        mult=[]
        
    elif option ==4: #Collection surface reflectance 500 m

        sel_col='MODIS/006/MOD09GA'
        bands = ['sur_refl_b01', 'sur_refl_b02','sur_refl_b03','sur_refl_b04','sur_refl_b05','sur_refl_b06','sur_refl_b07'] 
        #Â¡p_out='UIB-Imatges-MOD09GQ'
        p_out='UIB-Imatges-MOD09GA'
        #Image offset
        offset=10000
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA='MODIS/006/MOD09GA'
        QA_bands = ['state_1km', 'QC_500m']
        Indices=['NDVI','EVI2']
        scale=500
        mult=[]

    elif option ==5: #Collection Terra land surface temperature 1000 m

        sel_col='MODIS/006/MOD11A1'
        bands = ['LST_Day_1km', 'Day_view_time','Day_view_angle','LST_Night_1km','Night_view_time','Night_view_angle','Emis_31','Emis_32'] 
        p_out='UIB-Imatges-MOD11A1'
        #Image offset for LST, view time and emissivity
        mult=[0.02,0.1,1,0.002,0.0005]
        offset=[0,0,-65,0.49,0]
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA='MODIS/006/MOD11A1'
        QA_bands = ['QC_Day', 'QC_Night']
        Indices=[]
        scale=1000

    elif option ==6: #Collection Aqua land surface temperature 1000 m

        sel_col='MODIS/006/MYD11A1'
        bands = ['LST_Day_1km', 'Day_view_time','Day_view_angle','LST_Night_1km','Night_view_time','Night_view_angle','Emis_31','Emis_32'] 
        p_out='UIB-Imatges-MYD11A1'
        #Image offset for LST, view time and emissivity
        mult=[0.02,0.1,1,0.002,0.0005]
        offset=[0,0,-65,0.49,0]
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA='MODIS/006/MOD11A1'
        QA_bands = ['QC_Day', 'QC_Night']
        Indices=[]
        scale=1000


    elif option ==7: #Collection LAndsat-8 OLI Tier 1
    
        #https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR
        sel_col='LANDSAT/LC08/C01/T1_SR'
        #bands = ['B2','B3','B4','B5','B6','B7','pixel_qa']
        bands = ['B4','B5','pixel_qa']
        p_out='Landsat8_NDVI'
        #Image offset for LST, view time and emissivity
        mult=[0.0001,0.0001,0.0001,0.0001,0.0001,0.0001]
        offset=[0,0,0,0,0,0]
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA=''
        QA_bands = ['pixel_qa']
        #Indices=['NDVI','EVI2','NDWI2','NDSI','Snow','Clouds']
        Indices=['NDVI']
        scale=30
        cor_rad=0.05 #this is to correct band red due to errors  in atmosferic correction. see https://labo.obs-mip.fr/multitemp/using-ndvi-with-atmospherically-corrected-data/ 

    elif option ==8: #Collection LAndsat-7 ETM+ Tier 1
    
        #https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T2_SR#bands        
        sel_col='LANDSAT/LE07/C01/T1_SR'
        bands = ['B1','B2','B3','B4','B5','B7','pixel_qa'] 
        p_out='UIB-Imatges-Landsat-7-Ref'
        #Image offset for LST, view time and emissivity
        mult=[0.0001,0.0001,0.0001,0.0001,0.0001,0.0001]
        offset=[0,0,0,0,0,0]
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA=''
        QA_bands = ['pixel_qa']
        Indices=['NDVI','EVI2','NDWI2','NDSI','Snow','Clouds']
        Indices=['NDVI','EVI2','NDWI2']
        scale=30
        cor_rad=0.05 #this is to correct band red due to errors  in atmosferic correction. see https://labo.obs-mip.fr/multitemp/using-ndvi-with-atmospherically-corrected-data/ 


    elif option ==9: #Collection LAndsat-5 TM Tier 1
    
        #https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C01_T2_SR
        sel_col='LANDSAT/LT05/C01/T1_SR'
        bands = ['B1','B2','B3','B4','B5','B7','pixel_qa'] 
        p_out='UIB-Imatges-Landsat-5-Ref'
        #Image offset for LST, view time and emissivity
        mult=[0.0001,0.0001,0.0001,0.0001,0.0001,0.0001]
        offset=[0,0,0,0,0,0]
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA=''
        QA_bands = ['pixel_qa']
        Indices=['NDVI','EVI2','NDWI2','NDSI','Snow','Clouds']
        Indices=['NDVI','EVI2','NDWI2']
        scale=30
        cor_rad=0.05 #this is to correct band red due to errors  in atmosferic correction. see https://labo.obs-mip.fr/multitemp/using-ndvi-with-atmospherically-corrected-data/ 


    elif option ==10: #Collection Landsat-4 TM Tier 2
    
        #https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT04_C01_T2_SR
        sel_col='LANDSAT/LT04/C01/T2_SR'
        bands = ['B1','B2','B3','B4','B5','B7','pixel_qa'] 
        p_out='UIB-Imatges-Landsat-4-Ref'
        #Image offset for LST, view time and emissivity
        mult=[0.0001,0.0001,0.0001,0.0001,0.0001,0.0001]
        offset=[0,0,0,0,0,0]
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA=''
        QA_bands = ['pixel_qa']
        Indices=['NDVI','EVI2','NDWI2','NDSI','Snow','Clouds']
        scale=30
        cor_rad=0.05 #this is to correct band red due to errors  in atmosferic correction. see https://labo.obs-mip.fr/multitemp/using-ndvi-with-atmospherically-corrected-data/ 

    elif option ==11: #Collection LAndsat-8 OLI Tier 2
    
        #https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T2_SR
        sel_col='LANDSAT/LC08/C01/T2_SR'
        bands = ['B2','B3','B4','B5','B6','B7','pixel_qa']
        #bands = ['pixel_qa']
        p_out='UIB-Imatges-Landsat-8-Ref_T2'
        #Image offset for LST, view time and emissivity
        mult=[0.0001,0.0001,0.0001,0.0001,0.0001,0.0001]
        offset=[0,0,0,0,0,0]
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA=''
        QA_bands = ['pixel_qa']
        Indices=['NDVI','EVI2','NDWI2','NDSI','Snow','Clouds']
        scale=30
        cor_rad=0.05 #this is to correct band red due to errors  in atmosferic correction. see https://labo.obs-mip.fr/multitemp/using-ndvi-with-atmospherically-corrected-data/ 

    elif option ==12: #Collection LAndsat-7 Tier 2
    
        sel_col='LANDSAT/LE07/C01/T2_SR'
        bands = ['B2','B3','B4','B5','B6','B7','pixel_qa']
        #bands = ['pixel_qa']
        p_out='UIB-Imatges-Landsat-7-Ref_T2'
        #Image offset for LST, view time and emissivity
        mult=[0.0001,0.0001,0.0001,0.0001,0.0001,0.0001]
        offset=[0,0,0,0,0,0]
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA=''
        QA_bands = ['pixel_qa']
        Indices=['NDVI','EVI2','NDWI2','NDSI','Snow','Clouds']
        scale=30
        cor_rad=0.05 #this is to correct band red due to errors  in atmosferic correction. see https://labo.obs-mip.fr/multitemp/using-ndvi-with-atmospherically-corrected-data/ 
        
    elif option ==13: #Collection LAndsat-5 Tier 2
    
        sel_col='LANDSAT/LT05/C01/T2_SR'
        bands = ['B2','B3','B4','B5','B6','B7','pixel_qa']
        #bands = ['pixel_qa']
        p_out='UIB-Imatges-Landsat-5-Ref_T2'
        #Image offset for LST, view time and emissivity
        mult=[0.0001,0.0001,0.0001,0.0001,0.0001,0.0001]
        offset=[0,0,0,0,0,0]
        #Select MODIS product where QA bands is and the QA bands to be processed from the 500m product
        QA=''
        QA_bands = ['pixel_qa']
        Indices=['NDVI','EVI2','NDWI2','NDSI','Snow','Clouds']
        scale=30        
        cor_rad=0.05 #this is to correct band red due to errors  in atmosferic correction. see https://labo.obs-mip.fr/multitemp/using-ndvi-with-atmospherically-corrected-data/ 
        
    #parameters are returned
    set_par = set_par_est(sel_col =sel_col, bands=bands, p_out=p_out, offset=offset, QA=QA, QA_bands=QA_bands, Indices=Indices, scale=scale, mult=mult)

    return set_par


In [ ]:
def accumulate(image,img):
  name_image = image.get('system:index')
  image = image.select([0],[name_image])
  cumm = ee.Image(img).addBands(image)
  return cumm


def CloudMask(qa):
    
     #https://github.com/profLewis/getModisEE/blob/master/getModisEE/mapsModisEE.py
     #https://sryhandiniputeri.medium.com/the-difference-of-filtering-clouds-and-masking-cloud-in-google-earth-engine-260744bcc600
     #https://stackoverflow.com/questions/55040184/qa-with-different-bits-in-gee
     # Bits 4 is snow.
     cloudShadowBitMask=(1 << 3)
     cloudsBitMask=(1 << 5)
     
     #Get the pixel QA band.
     #Both flags should be set to 1, indicating clear conditions. 0 is clouds+clouds shadows and 1 the rest
     mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0)) 
     
     
     
     #Turn mask 1- clouds+shadows and 0 -OK values
     #mask=mask.Not().multiply(-10000).add(1)
     #mask=mask.Not().multiply(-10000).add(1)

     
     #mask = getQABits(qa,4,4,'snow_detected').eq(1)

     #mask_c = getQABits(qa,5,5,'cloud')
     
     
     #mask_c_s=getQABits(qa,3,3,'cloud_shadows')


     #mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))

     #mask = qa.bitwiseAnd(cloudsBitMask).eq(0)

    

     #sombra = getQABits(qa,3,3,'cloud_shadow')
     #nubes = getQABits(qa,5,5,'cloud')
    
     #Return an image masking out cloudy areas.
     #return qa.updateMask(sombra.eq(0)).updateMask(nubes.eq(0))

    
     #mask=((mask_c.eq(0)).And(mask_c_s.eq(0)))

     #mask=(mask_c.eq(0))
     

     return mask
 
    #mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)

    #mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)

#    return image.updateMask(sombra.eq(0)).updateMask(nubes.eq(0).updateMask(cirrus_detected.eq(0))).updateMask(snow_detected.eq(0))

In [ ]:
#en aquest mòdul eliminarem els pí­xels que tenen errors d'adquisició i aplicarem una màscara de núvols (sembla bàsicament per modis)
#https://gis.stackexchange.com/questions/308456/get-a-mask-for-modis-250m-mod09gq-using-modis-500m-mod09ga-in-google-earth
#https://gis.stackexchange.com/questions/363929/how-to-apply-a-bitmask-for-radiometric-saturation-qa-in-a-image-collection-eart

def CloudErrorsMask(QA,option,type_QA):

        #Valus is the actual image, QA_1 and QA_2 are the QA band and option depends on the product

    if option ==3:
        
        if type_QA==1:
            
            #state_1km
            #Bit 10 is the actual cloud mask    
            CloudsBitMask = 1 << 10
            #return value.updateMask(QA.bitwiseAnd(CloudsBitMask).eq(0))
            return QA.bitwiseAnd(CloudsBitMask).eq(0)

    elif option ==5: #for LST

        if type_QA==1: 
            
            #state_1km
            #Bit 10 is the actual cloud mask    
            CloudsBitMask = 2 << 0
            #return value.updateMask(QA.bitwiseAnd(CloudsBitMask).eq(0))
            #https://stackoverflow.com/questions/55040184/qa-with-different-bits-in-gee

            return QA.bitwiseAnd(3).eq(2)

            #return QA.bitwiseAnd(CloudsBitMask).eq(2)

        """
        if type_QA==2:

            #QC_500m
            # Bits 0 and 1 are give information about possible erros in bands.
            ErrorBitMask = 2 << 0

            #return value.updateMask(QA.bitwiseAnd(CloudsBitMask).eq(0))
            return QA.bitwiseAnd(CloudsBitMask).eq(0)

        """
        
        #Bit 8-9 is the cirrus mask. Values 01 and 10, thus 1 and 2, are going to be considered differently
        #for the timesat mask. Thus another code needs to be applied in the final mask
        #CirrusBitMask = 2 << 8


In [ ]:
#Main section
if __name__ == "__main__": 
      
    
    #Select option to download and pre-process imagery. See def CollectionSelectionPar(option) function for more details
    option=7 #la 11 és de Landsat8-tier2-SR
    
    #Define specific Landsat Path and Row
    #Mallorca 196-197 ==> 32 i 33
    #Plana Lleida 198/31

    path_l=198
    row_l=31

    path_l=""
    row_l=""


    #option parameters    
    par=CollectionSelectionPar(option)
    
    #If path and row selectors activated then output folder name is changes
    if path_l!="":
    
        path_out=par.p_out+"_"+str(path_l)+"_"+str(row_l)
    
    else:
        
        path_out=par.p_out


In [ ]:
    # https://www.usgs.gov/core-science-systems/nli/landsat/landsat-satellite-missions?qt-science_support_page_related_con=0#qt-science_support_page_related_con
    
    #Time extent
    #time=["2000-01-01","2020-12-31"]
    #time=["2000-04-30","2000-05-01"]
    #time=["2000-01-01","2020-01-01"]
    

    #Landsat-3 
    #time=["1978-03-01","1983-09-30"]

    #Landsat-2 
    #time=["1975-01-01","1983-07-30"]
    	
    #Landsat-1 
    #time=["1972-07-01","1978-01-30"]
	

    #Landsat-7 
    #time=["1999-04-01","2020-11-30"]
    #time=["2011-09-10","2020-11-30"]

    #Landsat-4 
    #time=["1982-07-01","1993-12-31"]


	 #Landsat-5 
    #time=["1984-03-01","2013-06-05"]
    #time=["2001-10-08","2013-06-05"]

    #Landsat-8 des de 2013
    #time=["2013-03-31","2020-12-31"]
    #time=["2020-11-22","2020-12-31"]


    #time=["1984-03-01","2013-06-05"]
    #time=["2001-10-08","2013-06-05"]
    #time=["2011-08-09","2013-06-05"] #Andratx
    #time=["2013-06-05","2020-12-31"] #Andratx L7
    time=["2015-01-01","2018-12-31"] #estèpiques Lleida


    #Time interval
    time_interval=1

    #compute time interval list
    t_period=ComputeProcessingPeriods(time, time_interval)
    
    #landsat = ee.ImageCollection("MODIS/006/MOD09GQ")
    
    # filter area
    #landsat_AOI = landsat.filterBounds(Mont_AOI)


Number of days to be processed: 1461
Number of processing periods: 1461.0
Time series is regular: total number of processing periods= 1461.0


Run the ee.Authenticate function to authenticate your access to Earth Engine servers and ee.Initialize to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()
####aquestes dues línies veien per defecte, a continuació el codi del Jordi per córrer amb python (no colab?)

#Check EE initialization
#    try:
#      ee.Initialize()
#      print('The Earth Engine package initialized successfully!')
#    except ee.EEException as e:
#      print('The Earth Engine package failed to initialize!')
#    except:
#      print("Unexpected error:", sys.exc_info()[0])
#      raise

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=kC5LYfx1H5Yja6_fevU1vkdqdkRY6gv6GIS8X6hI14A&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6zPAXiCSaCXrQ29O2EejwCsagYvEdSxOQC3fHCREqJ7G9lgygQnnI

Successfully saved authorization token.


Definir les àrees d'interès

In [ ]:

    # setting the Area of Interest (AOI) ==> Malllorca
    #pt = ee.Geometry.Rectangle([2.22670831, 40.00861727,3.55616778, 39.23497596])
    
    #Mallorca
    #pt = ee.Geometry.Rectangle([2.22670831, 40.00861727,3.55616778, 39.23497596])

    #Montseny
    #pt = ee.Geometry.Rectangle([2.24090420, 41.86440526,2.56749104, 41.67695256])


    # setting the Area of Interest (AOI) ==> Estèpiques
pt = ee.Geometry.Rectangle([0.31, 41.34,1.43, 41.90])



region = pt.buffer(10)
    
    
    
    ###NO HABILITAT ##########
    #Mallorca
    #pt = ee.Geometry.Rectangle([434000, 4429000, 548000, 4343000])
    
    #Conca Andratx
    #pt = ee.Geometry.Rectangle([447500, 4384600, 450900, 4380500])
    
    #Montseny
    #spatial extent for exporting
    #pt = ee.Geometry.Rectangle([437000, 4635000, 464000, 4614000])
    #region_exp = pt.buffer(10)
    ###NO HABILITAT ##########



    #load Mallorca boundaries for image cliping
    #mall_mask = ee.FeatureCollection('users/jcristobalrossello/Cartografia/Mallorca_mask')
    

    #load Montseny boundaries for image masking
    #LimitsParc = ee.FeatureCollection('users/jcristobalrossello/Cartografia/LimitsParc')


    #load Montseny water bodies for image masking
    #MassesAigua = ee.FeatureCollection('users/jcristobalrossello/Cartografia/MassesAigua')


                        
    #Make an image out of the land area attribute.
   
    # Make an image out of the population attribute and display it.
    #popImage = mall_mask.filter(ee.Filter.neq('CODI', None))\
    #                     .reduceToImage(**{'properties': ['CODI'],
    #                                       'reducer': ee.Reducer.first()})


    #apply mask
    #Select the land/water mask.
    #datamask = popImage.eq(1)

    #apply mask
    #Select the land/water mask.
    #water_mask = popImage.eq(1).mask().Not()


In [ ]:
    #Here we loop through.0 the temporal extent
for t_per in t_period:
    
        if path_l=="":
    
            col = (ee.ImageCollection(par.sel_col)
                          #.filterDate(datetime.datetime(2018, 7, 1),datetime.datetime(2018, 7, 1))
                          .filterDate(t_per[0],t_per[1])
                          .filterBounds(region)
                          .select(par.bands)
                          .sort('system:time_start')
                      )
    
        else:
    
            col = (ee.ImageCollection(par.sel_col)
                          #.filterDate(datetime.datetime(2018, 7, 1),datetime.datetime(2018, 7, 1))
                          .filterDate(t_per[0],t_per[1])
                          .filterBounds(region)
                          .select(par.bands)
                          .sort('system:time_start')
                          .filter(ee.Filter.eq('WRS_PATH', path_l))
                          .filter(ee.Filter.eq('WRS_ROW', row_l))
                          #.filterMetadata('WRS_PATH', 'equals', path_l)
                          #.filterMetadata('WRS_ROW', 'equals', row_l)
                      )
        
        #Here we check if there are images 
        if col.size().getInfo() == 0:

            print('Image not found for:',t_per[0])      


        else:    
        
            print('Processing image:',t_per[0])    
        
        
        
            if option==3 or option==4:
    
                  
                #Mask processing
                QA = (ee.ImageCollection(par.QA)
                              #.filterDate(datetime.datetime(2018, 7, 1),datetime.datetime(2018, 7, 1))
                              .filterDate(t_per[0],t_per[1])
                              .filterBounds(region)
                              .select(par.QA_bands)
                              .sort('system:time_start'))
            
            
    
                #Here the 500m product is combined to the 250 to extraxt the 500m QA bands            
                #col = ee.ImageCollection("MODIS/006/MOD09GA")
                #combined = p_500m.combine(col).filterDate(t_per[0],t_per[1])
        
                
                
                #We search day by day, so it has to be 1 image per query
                if col.size().getInfo() == 1:
            
                    date = ee.Date (col.first().get('system:time_start'))
            
                    time = date.getInfo()['value']/1000.
                    
                    #Here is the image date used as suffix    
                    suffix=dt.utcfromtimestamp(time).strftime('%Y%m%d')+'_'+str(par.scale)+'m'
        
                    bands = par.bands
    
    
                    if option==4: #clouds are only applied to the 500m product
        
                        QA_bands=par.QA_bands
                    
                        #Masks are extracted
                        for QA_band in QA_bands:
                            
                          QA_band_sel = QA.map(lambda img: img.select(QA_band)\
                                                       .set('system:time_start', img.get('system:time_start'))\
                                                       .set('system:index', img.get('system:index')))
                          #  ImageCollection to List           
                          QA_list = QA_band_sel.toList(QA_band_sel.size())
                        
                          #  Define the initial value for iterate.
                          QA_base = ee.Image(QA_list.get(0))
                          QA_base_name = QA_base.get('system:index')
                          QA_base = QA_base.select([0], [QA_base_name])
                          
                          #  Eliminate the image 'base'.
                          QA_new_col = ee.ImageCollection(QA_list.splice(0,1))
                        
                          QA_img_cummulative = ee.Image(QA_new_col.iterate(accumulate,QA_base))
        
        
                          if QA_band ==  'state_1km':
        
                            type_QA=1
        
                            CM=CloudErrorsMask(QA_img_cummulative,3,type_QA)
                    
                    
                       
                    #LST bands processing
                    for band in bands:
                     
                      col_band = col.map(lambda img: img.select(band)\
                                                   .set('system:time_start', img.get('system:time_start'))\
                                                   .set('system:index', img.get('system:index')))
                      #  ImageCollection to List           
                      col_list = col_band.toList(col_band.size())
                    
                      #  Define the initial value for iterate.
                      base = ee.Image(col_list.get(0))
                      base_name = base.get('system:index')
                      base = base.select([0], [base_name])
                      
                      #  Eliminate the image 'base'.
                      new_col = ee.ImageCollection(col_list.splice(0,1))
                      
                      """
                      if option==3:
                          
                          CM=CM.clip(region).reproject(crs='EPSG:32631', scale=par.scale)
                      """
                      #Apply error and cloud mask as well as study area mask + clip+ reprojection + offset
                      
                      
                      if option==4: #clouds are only applied to the 500m product
                          
                          img_cummulative = ee.Image(new_col.iterate(accumulate,base)).updateMask(datamask).updateMask(CM).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale).divide(par.offset)
                      
                      else:
                       
                          img_cummulative = ee.Image(new_col.iterate(accumulate,base)).updateMask(datamask).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale).divide(par.offset)
                      
                      
                      if band ==  'sur_refl_b01':
                          
                          #extreme values > or <0 are masked out before indices computation
                          mask_ev=ee.Image(1).where(img_cummulative.lte(0),0).where(img_cummulative.gt(1),0)
                          mask_ev=mask_ev.eq(1)
                          red=img_cummulative.updateMask(mask_ev)
                      
                      elif band ==  'sur_refl_b02':
                
                          #extreme values > or <0 are masked out before indices computation
                          mask_ev=ee.Image(1).where(img_cummulative.lte(0),0).where(img_cummulative.gt(1),0)
                          mask_ev=mask_ev.eq(1)
                          nir=img_cummulative.updateMask(mask_ev)
                          #Here we compute the vegetation indices
                    for Index in par.Indices:
    
                        if Index ==  'NDVI':
                          
                          #image = nir.subtract(red).divide(nir.add(red)).resample('bilinear').reproject(crs='EPSG:32631', scale=250).float()
                          image = nir.subtract(red).divide(nir.add(red)).float()
        
                        elif Index ==  'EVI2':
                          
                          
                          #Computed from https://www.indexdatabase.de/db/i-single.php?id=237
                          #"""EVI2 = image.expression(
                          #       '2.4 * ((NIR - RED) / (NIR + RED + 1))', {
                          #           'NIR': nir,
                          #           'RED': red,
                          #          }).reproject(crs='EPSG:32631', scale=250).select('image2')
                          #  """     
                          #A cast to float is need to save it to 32 bit
                          #image=nir.subtract(red).multiply(2.4).divide(nir.add(red).add(1)).resample('bilinear').reproject(crs='EPSG:32631', scale=250).float()
                          image=nir.subtract(red).multiply(2.4).divide(nir.add(red).add(1)).float()
                        
    
                        #Index is exported                  
                        task = ee.batch.Export.image.toDrive(
                           image = image,
                          #folder = 'UIB/Imatges/MODIS/MOD09GQ/',
                           folder = path_out,
                           fileNamePrefix = Index+'_'+suffix,
                           fileFormat = 'GeoTIFF',
                           region=region,
                           ).start()  
                      
                        print('Export Image '+ Index+ ' was submitted, please wait ...') 
                
                
            elif (option==5) or (option==6): #for LST<
                
                #Mask processing
                QA = (ee.ImageCollection(par.QA)
                              #.filterDate(datetime.datetime(2018, 7, 1),datetime.datetime(2018, 7, 1))
                              .filterDate(t_per[0],t_per[1])
                              .filterBounds(region)
                              .select(par.QA_bands)
                              .sort('system:time_start'))
                                #Here the 500m product is combined to the 250 to extraxt the 500m QA bands            
                #col = ee.ImageCollection("MODIS/006/MOD09GA")
                #combined = p_500m.combine(col).filterDate(t_per[0],t_per[1])
        
                
                
                #We search day by day, so it has to be 1 image per query
                if col.size().getInfo() == 1:
            
                    date = ee.Date (col.first().get('system:time_start'))
            
                    time = date.getInfo()['value']/1000.
                    
                    #Here is the image date used as suffix    
                    suffix=dt.utcfromtimestamp(time).strftime('%Y%m%d')+'_'+str(par.scale)+'m'
        
        
                    bands = par.bands
                    QA_bands=par.QA_bands
                
    
                    #Masks are extracted
                    for QA_band in QA_bands:
                        
                      QA_band_sel = QA.map(lambda img: img.select(QA_band)\
                                                   .set('system:time_start', img.get('system:time_start'))\
                                                   .set('system:index', img.get('system:index')))
                      #  ImageCollection to List           
                      QA_list = QA_band_sel.toList(QA_band_sel.size())
                    
                      #  Define the initial value for iterate.
                      QA_base = ee.Image(QA_list.get(0))
                      QA_base_name = QA_base.get('system:index')
                      QA_base = QA_base.select([0], [QA_base_name])
                      
                      #  Eliminate the image 'base'.
                      QA_new_col = ee.ImageCollection(QA_list.splice(0,1))
                    
                      QA_img_cummulative = ee.Image(QA_new_col.iterate(accumulate,QA_base))
    
    
                      if QA_band ==  'QC_Day':
    
                        type_QA=1
    
                        CM_day=CloudErrorsMask(QA_img_cummulative,option,type_QA)
                        
                      elif QA_band ==  'QC_Night':
    
                        type_QA=1
    
                        CM_night=CloudErrorsMask(QA_img_cummulative,option,type_QA)
                    
                       
                    #LST bands
                    for band in bands:
                     
                     
                      col_band = col.map(lambda img: img.select(band)\
                                                   .set('system:time_start', img.get('system:time_start'))\
                                                   .set('system:index', img.get('system:index')))
                      #  ImageCollection to List           
                      col_list = col_band.toList(col_band.size())
                    
                      #  Define the initial value for iterate.
                      base = ee.Image(col_list.get(0))
                      base_name = base.get('system:index')
                      base = base.select([0], [base_name])
                      
                      #  Eliminate the image 'base'.
                      new_col = ee.ImageCollection(col_list.splice(0,1))
                      
                      #A different mask is apply depending if it is night or day                     
                      if band== 'LST_Day_1km' or band== 'Day_view_time' or band== 'Day_view_angle' or band== 'Emis_31' or band== 'Emis_32':
                          
                          CM=CM_day
                          
                      elif band== 'LST_Night_1km' or band== 'Night_view_time' or band== 'Night_view_angle':   
    
                          CM=CM_night
    
    
                      #Apply error and cloud mask as well as study area mask + clip+ reprojection + offset
                      #img_cummulative = ee.Image(new_col.iterate(accumulate,base)).updateMask(datamask).updateMask(CM).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale)
                      img_cummulative = ee.Image(new_col.iterate(accumulate,base)).updateMask(datamask).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale)
                      
                      
                      if band ==  'LST_Day_1km' or band ==  'LST_Night_1km':
                          
                          mult=par.mult[0]
                          offset=par.offset[0]
                          
                          image=img_cummulative.multiply(mult).add(offset).float()
                      
                      elif band ==  'Day_view_time' or band ==  'Night_view_time':
    
                          mult=par.mult[1]
                          offset=par.offset[1]
    
                          image=img_cummulative.multiply(mult).add(offset).float()
                    
                      elif band ==  'Day_view_angle' or band ==  'Night_view_angle':
    
                          mult=par.mult[2]
                          offset=par.offset[2]
    
                          image=img_cummulative.multiply(mult).add(offset).float()
                    
    
                      elif band ==  'Emis_31':
                
                          mult=par.mult[3]
                          offset=par.offset[3]
                          image=img_cummulative.multiply(mult).add(offset).float()
                          emis_31=image.multiply(0.4587)
    
                      elif band ==  'Emis_32':
    
                          mult=par.mult[3]
                          offset=par.offset[3]
                          image=img_cummulative.multiply(mult).add(offset).float()
                          emis_32=image.multiply(0.5414)
    
    
                      elif band ==  'Clear_day_cov' or band ==  'Clear_night_cov':
                
                          mult=par.mult[4]
                          offset=par.offset[4]
                          image=img_cummulative.multiply(mult).add(offset).float()
    
    
    
                      #Index is exported                  
                      task = ee.batch.Export.image.toDrive(
                           image = image,
                           folder = par.p_out,
                           fileNamePrefix = band+'_'+suffix,
                           fileFormat = 'GeoTIFF',
                           region=region,
                           ).start()  
    
                      print('Export Image '+ band+ ' was submitted, please wait ...') 
    
                    #Emis_broad is computed
                    #CristÃ³bal et al. Data paper 2017
                    image= emis_31.add(emis_32).float()
                      
                    #Index is exported                  
                    task = ee.batch.Export.image.toDrive(
                         image = image,
                         folder = path_out,
                         fileNamePrefix = 'Emis_broad_'+suffix,
                         fileFormat = 'GeoTIFF',
                         region=region,
                         ).start()  
    
                    print('Export Image Emis_broad was submitted, please wait ...') 
    
    
            elif (option==7) or (option==8)or (option==9) or (option==10): #for Landsat-4 to 8
               
                
               #Multipliers and offset are the same for all Landsat optical band products
                mult=par.mult[0]
                offset=par.offset[0]
                cor_rad=par.mult[0]
                

                #Before explorting the imagery clouds, coud shadows and snow cover is maked out in addition to the Mallorca mask
                #col=col.map(maskQuality)
               
                
                #We search day by day, so it has to be 1 image per query
                if col.size().getInfo() >= 1: #more than one consecutive Landsat image from the same path but different rows. Mosaiking is needed
                    #https://developers.google.com/earth-engine/guides/ic_composite_mosaic

                    """
                    image=ee.Image(col.mosaic())
                    
                    #https://gis.stackexchange.com/questions/280156/mosaicking-a-image-collection-by-date-day-in-google-earth-engine
                    
                    #Add the mosaic to a list only if the collection has images
                    day_mosaics=ee.List(image)

                    col = ee.ImageCollection(ee.List(day_mosaics))
                    """
                    #print(col.size().getInfo())
                    
                    date_i=[t_per[0].format('YYYY-MM-DD')]    
                    
                    #Here is the image date used as suffix    
                    #suffix=dt.utcfromtimestamp(time).strftime('%Y%m%d')+'_'+str(par.scale)+'m'
                    suffix=date_i[0].replace("-","")+'_'+str(par.scale)+'m'
        
        
                    bands = par.bands
                
                    #Landsat bands
                    for band in bands:
                     
                      
                     if band != 'pixel_qa_':
                        
                          col_band = col.map(lambda img: img.select(band)\
                                                       .set('system:time_start', img.get('system:time_start'))\
                                                       .set('system:index', img.get('system:index')))
                          #ImageCollection to List           
                          col_list = col_band.toList(col_band.size())
                        
                          #  Define the initial value for iterate.
                          base = ee.Image(col_list.get(0))
                          base_name = base.get('system:index')
                          base = base.select([0], [base_name])
                          
                          #  Eliminate the image 'base'.
                          #new_col = ee.ImageCollection(col_list.splice(0,1))
                          new_col = ee.ImageCollection(col_list.splice(0,1))

                          
                          #Masks are applied and image is clipped. NODATA values as masked as -9999
                          #img_cummulative = ee.Image(new_col.iterate(accumulate,base)).updateMask(datamask).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale).unmask(-9999)

                          #If there are more than 2 scenes in the dataset we reduce the image to one, similar to mosaic  
                          if col.size().getInfo() >1:

                              img_cummulative = ee.Image(new_col.iterate(accumulate,base)).reduce(ee.Reducer.mean())

                          else:
                               
                              img_cummulative = ee.Image(new_col.iterate(accumulate,base))
                          

                          #img_cummulative = ee.Image(new_col.iterate(accumulate,base)).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale).unmask(-9999)
                          
                          
                          #Image selection for Landsat time series    
                          if option!=7 and band=='B4':
                              
                              nir=img_cummulative.multiply(mult).add(offset).float()
        
                          elif option!=7 and band=='B7':
        
                              swir=img_cummulative.multiply(mult).add(offset).float()
        
                          elif option!=7 and band=='B3':
                                  
                              red=img_cummulative.multiply(mult).add(offset).float()
        
                          elif option!=7 and band=='B2':
                                  
                              green=img_cummulative.multiply(mult).add(offset).float()
     
                          elif option!=7 and band=='B5':
                                  
                              swir_1=img_cummulative.multiply(mult).add(offset).float()
        
                                
                          elif option==7: #for landsat 8    
                            
                            if band=='B4': #red                       
        
                              red=img_cummulative.multiply(mult).add(offset).add(cor_rad).float()
                            
                            elif band=='B5':   #nir                     
        
                              nir=img_cummulative.multiply(mult).add(offset).float()
        
                            elif band=='B7': #swir
        
                              swir=img_cummulative.multiply(mult).add(offset).float()
        

                            elif band=='B6': #swir 1
        
                              swir_1=img_cummulative.multiply(mult).add(offset).float()

                            elif band=='B3': #green
        
                              green=img_cummulative.multiply(mult).add(offset).float()
        
                          if band != 'pixel_qa':

                              #Apply image offset for all bands                     
                              image=img_cummulative.multiply(mult).add(offset)
                              
                              #image=img_cummulative
                              
                          elif band == 'pixel_qa':

                              image=img_cummulative
                              
                              qa=img_cummulative

                          else: 
                              
                              
                              #Export snow:
                              #image=SnowMask(img_cummulative)
                              
                              #Apply image offset for all bands                     
                              #image=img_cummulative
                              image=image.updateMask(datamask).clip(region).resample('bilinear').reproject(crs='EPSG:32631', scale=par.scale).unmask(-9999)

                        #Image is exported                  
                          task = ee.batch.Export.image.toDrive(
                               image = image,
                               folder = path_out,
                               fileNamePrefix = band+'_'+suffix,
                               fileFormat = 'GeoTIFF',
                               region=region,
                               ).start()  
        
                          print('Export Image '+ band+ ' was submitted, please wait ...') 
        
                           
                    #Here we compute the vegetation indices
                    for Index in par.Indices:
    
                        if Index ==  'NDVI':
                          
                          #image = nir.subtract(red).divide(nir.add(red)).resample('bilinear').reproject(crs='EPSG:32631', scale=250).float()
                          image = nir.subtract(red).divide(nir.add(red)).float()

                        elif Index ==  'Clouds': #the cloud+cloud shadows mask is created. 1 for clouds+shadows; 0 the rest
                            
                            c_mask=CloudMask(qa)

                            #Turn mask 1- clouds+shadows and 0 -OK values
                            image=c_mask.Not()


                        else:
                            
                            print(Index+ ' not implemented yet. ...') 
                            
    
                        #Index is exported                  
                        task = ee.batch.Export.image.toDrive(
                           image = image,
                          #folder = 'UIB/Imatges/MODIS/MOD09GQ/',
                           folder = path_out,
                           fileNamePrefix = Index+'_'+suffix,
                           fileFormat = 'GeoTIFF',
                           region=region,
                           ).start()  
                      
                        print('Export Image '+ Index+ ' was submitted, please wait ...') 


Image not found for: 2015-01-01
Image not found for: 2015-01-02
Image not found for: 2015-01-03
Image not found for: 2015-01-04
Image not found for: 2015-01-05
Image not found for: 2015-01-06
Processing image: 2015-01-07
Export Image pixel_qa was submitted, please wait ...
Export Image NDVI was submitted, please wait ...
Image not found for: 2015-01-08
Image not found for: 2015-01-09
Image not found for: 2015-01-10
Image not found for: 2015-01-11
Image not found for: 2015-01-12
Image not found for: 2015-01-13
Processing image: 2015-01-14
Export Image pixel_qa was submitted, please wait ...
Export Image NDVI was submitted, please wait ...
Image not found for: 2015-01-15
Image not found for: 2015-01-16
Image not found for: 2015-01-17
Image not found for: 2015-01-18
Image not found for: 2015-01-19
Image not found for: 2015-01-20
Image not found for: 2015-01-21
Image not found for: 2015-01-22
Image not found for: 2015-01-23
Image not found for: 2015-01-24
Image not found for: 2015-01-25
Im

https://gis.stackexchange.com/questions/323921/google-earth-engine-mask-application
       
        
https://gis.stackexchange.com/questions/261495/filter-image-collection-with-a-date-list-in-google-earth-engine

https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/remove-clouds-from-landsat-data/
  
  
https://isp.uv.es/projects/cdc/GEE_cloud_detection_report.html

https://www.earthdatascience.org/courses/

https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/remove-clouds-from-landsat-data/

https://github.com/sacridini/GEET

https://medium.com/@sryhandiniputeri/the-difference-of-filtering-clouds-and-masking-cloud-in-google-earth-engine-260744bcc600
  
Sentinel 
https://foodsecurity-tep.net/S2_NDWI